In [1]:
import qdmpy.io as Qio
import qdmpy.fit as Qfit
import qdmpy.plot as Qplot
import qdmpy.field as Qfield

import matplotlib

In [2]:
# '%matplotlib widget' allows interactive plots in jupyterlab
# use ''%matplotlib inline' to allow export to pdf
%matplotlib widget

### Load raw data

In [3]:
options_dict = {
    "base_dir": "/home/samsc/ResearchData/test_images/"
    "filepath": "Spin relaxation (T1)_19",
    "custom_output_dir_suffix": "_{ROI}_bin_{total_bin}",
    
    "additional_bins": 0,

    "system_name": "Liams_Widefield",
    "ROI": "Full", 
    "ROI_start": [10, 10],
    "ROI_end": [200,200],
    
    "AOI_1_start": [10, 10],
    "AOI_1_end": [20, 20], 
    "AOI_2_start": [10, 50],
    "AOI_2_end": [20, 60], 
    "AOI_3_start": [60, 15],
    "AOI_3_end": [70, 35], 
    
    "normalisation": "div",
    
    "fit_pixels": True, 
    "force_fit": True, 
    "auto_match_prev_ROI_options": False,
    
    "fit_backend": "gpufit",
    "fit_backend_comparison": ["scipyfit", "gpufit"],
    
    "scipy_sub_threads": 2, 

    "fit_functions":  {"constant": 1, "stretched_exponential": 1},

    "constant_guess": 1,
    "constant_bounds": [0.9, 1.1],

    "power_exp_guess": 0.7,
    "power_exp_bounds": [0.5, 1.0],

    "charac_exp_t_guess": 1e-3,
    "charac_exp_t_bounds": [1e-9, 100e-3],

    "amp_exp_guess": 0.07,
    "amp_exp_bounds": [0.001, 0.2],
    
    "save_fig_type": "pdf",
    
    "calc_field_pixels": False,
    },
    "mpl_rcparams": {
        "legend.loc": "upper right" 
}
# if you want to use a reference experiment {ensure you run this even if 'None' as it sets up output dirs etc.}
ref_options_dir = "/home/samsc/ResearchData/Photovoltaics/2021-03-04_D298_SiPV-22_solo/ODMR - Pulsed without IR_15_Full_bin_16/"

In [4]:
options = Qio.load_options(options_dict=options_dict, check_for_prev_result=True)
Qplot.set_mpl_rcparams(options)

### Load data

In [5]:
image, prelim_sweep_list = Qio.load_image_and_sweep(options)
PL_image, PL_image_ROI, sig, ref, sig_norm, single_pixel_pl, sweep_list = Qio.reshape_dataset(
        options, image, prelim_sweep_list
    )
ROI_PL_fig = Qplot.plot_ROI_PL_image(options, PL_image)

AOI_PL_fig = Qplot.plot_AOI_PL_images(options, PL_image_ROI)

AOI_spectra_fig = Qplot.plot_AOI_spectra(options, sig, ref, sweep_list)

Qio.save_PL_data(options, PL_image, PL_image_ROI)

/home/samsc/src/nv/qdmpy_proj/qdmpy_git/qdmpy/io/raw.py:307: UserWarning: Avoiding IndexError for single_pixel_check (setting pixel check to centre of image):
index 150 is out of bounds for axis 1 with size 96
  warnings.warn(


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Load reference experiment - note, you can fit in this same notebook!

In [6]:
# below: alternative is to just have the dictionary already loaded (above this is called `pixel_fit_params`)
ref_fit_params, ref_sigmas = Qio.load_reference_experiment_fit_results(options, ref_options=None, ref_options_dir=ref_options_dir)

### Fit AOI, ROI averages, single pixel

In [7]:
fit_model = Qfit.define_fit_model(options)
backend_ROI_results_lst = Qfit.fit_ROI_avg(options, sig_norm, sweep_list, fit_model)
ROI_fit_fig = Qplot.plot_ROI_avg_fits(options, backend_ROI_results_lst)

fit_result_collection_lst = Qfit.fit_AOIs(options, sig_norm, single_pixel_pl, sweep_list, fit_model, backend_ROI_results_lst)
AOI_fit_fig = Qplot.plot_AOI_spectra_fit(options, sig, ref, sweep_list, fit_result_collection_lst, fit_model)

/home/samsc/src/nv/qdmpy_proj/qdmpy_git/qdmpy/fit/_scipyfit.py:35: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm  # auto detects jupyter


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Fit pixels

In [8]:
wanted_roi_result = next(
    filter(
        lambda result: result.fit_backend == options["fit_backend"], backend_ROI_results_lst
    )
) # ROI fit result for chosen fit backend
pixel_fit_params, sigmas = Qfit.get_pixel_fit_results(options, sig_norm, sweep_list, fit_model, wanted_roi_result)

Qio.save_pixel_fit_results(options, pixel_fit_params)
Qio.save_pixel_fit_sigmas(options, sigmas)

/home/samsc/src/nv/qdmpy_proj/qdmpy_git/qdmpy/fit/interface.py:281: UserWarning: Using previous fit results.
  warnings.warn("Using previous fit results.")


### Plot (fit) parameters

In [9]:
c_fig = Qplot.plot_param_images(options, fit_model, pixel_fit_params, "constant")
pow_fig = Qplot.plot_param_images(options, fit_model, pixel_fit_params, "power_exp")
exp_fig = Qplot.plot_param_images(options, fit_model, pixel_fit_params, "charac_exp_t")
amp_fig = Qplot.plot_param_images(options, fit_model, pixel_fit_params, "amp_exp")
residual_fig = Qplot.plot_param_images(options, fit_model, pixel_fit_params, "residual")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
pos_sigma_plot = Qplot.plot_param_images(options, fit_model, sigmas, "pos", errorplot=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Initial guess and fit vs data (flattened)

In [11]:
flat_pos_fig = Qplot.plot_params_flattened(options, fit_model, pixel_fit_params, wanted_roi_result, "pos", sigmas=sigmas, plot_bounds=False, plot_sigmas=False, errorevery=1000)
flat_resid_fig = Qplot.plot_params_flattened(options, fit_model, pixel_fit_params, wanted_roi_result, "residual", sigmas=sigmas)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Save and (optionally) view options dict

In [17]:
Qio.save_options(options)
# print(Qio.dict_to_json_str(options))